In [51]:
import numpy as np
import torch 
import torch.nn as nn
import torch.nn.functional as F
import regex as re

In [52]:
data= open("../datasets/llm/shakeshpere.txt",encoding="utf-8", mode ="r").read()
len(data)
bytesData = [x for x in data.encode(errors="replace")]
x =ord("a")
len(bytesData)

1115394

In [56]:
class BPETokenizer:
    def __init__(self, nmerges: int):
        self.nextByte: int= 256
        self.nmerges: int = nmerges
        pass
    
    def _get_counts(self,byteData:list):
        counts = {}
        for i,j in zip(byteData,byteData[1:]):
            counts[i,j]=counts.get((i,j),0)+1
        return counts

    def _do_merges(self,keys:tuple,token:int,byteData: list) -> list:
        i=0
        while i < (len(byteData)):
            i+=1
            if i < (len(byteData)-1) and byteData[i] == keys[0] and byteData[i+1] == keys[1]:
                byteData[i] = token
                byteData.pop(i+1)
                i+=1
        return byteData
    
    def train(self,text:str):
        self.bytesData = [x for x in text.encode(errors="replace")]
        self.merges_dict: dict = {}
        for x in range(self.nmerges):
            bp_counts = self._get_counts(self.bytesData)
            req_key = max(bp_counts.items(),key=lambda x:x[1])[0]
            self.merges_dict[req_key] = self.nextByte + x
            self.bytesData = self._do_merges(req_key,self.nextByte+x,self.bytesData)
        
        self.decode_dict = {v:k for k,v in self.merges_dict.items()}
        

    def encode(self,text:str) -> list:
        tokens = [x for x in text.encode(errors="replace")]
        i=0
        while i < (len(tokens)):
            
            if i < (len(tokens)-1) and self.merges_dict.get((tokens[i],tokens[i+1]),False):
                tokens[i] = self.merges_dict[(tokens[i],tokens[i+1])]
                tokens.pop(i+1)
                i+=1
            else:
                i+=1
        
        return tokens
                

    def decode(self,tokens:list) -> str:
        bytes = []
        i=0
        while i < (len(tokens)):
            
            if self.decode_dict.get(tokens[i],False):
                bytes.extend(self.decode_dict[tokens[i]])
                i+=1
            else:
                bytes.extend([tokens[i]])
                i+=1
        return "".join([chr(x) for x in bytes])

In [57]:
vocab_size = 500
merges = vocab_size - 256
tokenizer = BPETokenizer(merges)

In [58]:
tokenizer.train(data)

In [66]:
len(tokenizer.encode(data))

711614

In [63]:
tokenizer.decode(tokenizer.encode(data[:1000])) == data[:1000]

True

In [61]:
data[:1000]

"First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citizens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we become rakes: for the gods know I\nspeak this in hunger 

In [ ]:
tokenizer.decode(tokenizer.encode(data[:1000])) == data[:1000]

121

In [59]:
tokenizer.merges_dict

{(101, 32): 256,
 (116, 104): 257,
 (116, 32): 258,
 (115, 32): 259,
 (100, 32): 260,
 (44, 32): 261,
 (111, 117): 262,
 (101, 114): 263,
 (105, 110): 264,
 (121, 32): 265,
 (97, 110): 266,
 (58, 10): 267,
 (111, 114): 268,
 (111, 32): 269,
 (101, 110): 270,
 (10, 10): 271,
 (97, 114): 272,
 (32, 257): 273,
 (111, 110): 274,
 (108, 108): 275,
 (104, 97): 276,
 (44, 10): 277,
 (46, 271): 278,
 (105, 259): 279,
 (101, 115): 280,
 (121, 262): 281,
 (32, 115): 282,
 (116, 269): 283,
 (266, 260): 284,
 (111, 119): 285,
 (101, 97): 286,
 (32, 109): 287,
 (32, 119): 288,
 (111, 102): 289,
 (32, 104): 290,
 (264, 103): 291,
 (111, 109): 292,
 (32, 97): 293,
 (99, 104): 294,
 (257, 256): 295,
 (115, 116): 296,
 (32, 98): 297,
 (110, 111): 298,
 (105, 114): 299,
 (102, 268): 300,
 (118, 256): 301,
 (101, 261): 302,
 (105, 257): 303,
 (273, 256): 304,
 (115, 101): 305,
 (108, 105): 306,
 (84, 104): 307,
 (275, 32): 308,
 (114, 101): 309,
 (115, 258): 310,
 (97, 258): 311,
 (65, 110): 312,
 (73, 3